In [7]:
import gym
import sys
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy #Probalistic approch to also try some new paths from time to time
from rl.memory import SequentialMemory

# import necessary modules from keras
import keras
from keras.models import InputLayer
import matplotlib.pyplot as plt

In [2]:
from ale_py import ALEInterface, SDL_SUPPORT
import atari_py

ale = ALEInterface()
ale.setInt("random_seed", 123)

ale.loadROM(atari_py.get_game_path('breakout'))

In [27]:
if SDL_SUPPORT:
    ale.setBool("sound", True)
    ale.setBool("display_screen", True)

# Load the ROM file
#rom_file = sys.argv[0]
#ale.loadROM(rom_file)

# Get the list of legal actions
legal_actions = ale.getLegalActionSet()

In [28]:
print(legal_actions)

[<Action.NOOP: 0>, <Action.FIRE: 1>, <Action.UP: 2>, <Action.RIGHT: 3>, <Action.LEFT: 4>, <Action.DOWN: 5>, <Action.UPRIGHT: 6>, <Action.UPLEFT: 7>, <Action.DOWNRIGHT: 8>, <Action.DOWNLEFT: 9>, <Action.UPFIRE: 10>, <Action.RIGHTFIRE: 11>, <Action.LEFTFIRE: 12>, <Action.DOWNFIRE: 13>, <Action.UPRIGHTFIRE: 14>, <Action.UPLEFTFIRE: 15>, <Action.DOWNRIGHTFIRE: 16>, <Action.DOWNLEFTFIRE: 17>]


In [29]:
for episode in range(10):
    total_reward = 0
    while not ale.game_over():
        a = legal_actions[random.randrange(len(legal_actions))]
        # Apply an action and get the resulting reward
        reward = ale.act(a)
        total_reward += reward
    print("Episode %d ended with score: %d" % (episode, total_reward))
    ale.reset_game()

Episode 0 ended with score: 2
Episode 1 ended with score: 1
Episode 2 ended with score: 0
Episode 3 ended with score: 2
Episode 4 ended with score: 0
Episode 5 ended with score: 1
Episode 6 ended with score: 4
Episode 7 ended with score: 2
Episode 8 ended with score: 2
Episode 9 ended with score: 0


In [3]:
env = gym.make('Pong-v0')#endlich läuft es --endlich--

In [4]:
states = env.observation_space.shape[0]
actions = env.action_space.n

In [5]:
print('States:{}, Actions:{}'.format(states, actions))

States:210, Actions:6


In [9]:
#random inputs for visualisation
for episodes in range(10):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([2,3])
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episodes:{} Score:{}'.format(episodes, score))

C:\Users\erics\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\atari\environment.py:255: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(
C:\Users\erics\AppData\Local\Programs\Python\Python39\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Der Threadmodus kann nicht nach dem Einstellen geändert werden
  warnings.warn(str(err))


Episodes:0 Score:-21.0
Episodes:1 Score:-21.0
Episodes:2 Score:-20.0
Episodes:3 Score:-21.0
Episodes:4 Score:-21.0
Episodes:5 Score:-20.0
Episodes:6 Score:-19.0
Episodes:7 Score:-19.0
Episodes:8 Score:-20.0
Episodes:9 Score:-21.0


In [10]:

state = env.reset()
plt.imshow(state)

In [11]:
def prepro(I):
    # prepro 210x160x3 frame into 6400 (80x80) 1D float vector 
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(float).ravel()#Show preprocessedobs_preprocessed = prepro(observation).reshape(80,80)

plt.imshow(n_state, cmap='gray')

prepro_state = prepro(n_state).reshape(80,80)
plt.imshow(prepro_state, cmap='gray')

In [ ]:
def discount_rewards(r, gamma):
    # take 1D float array of rewards and compute discounted reward 
    r = np.array(r)
    discounted_r = np.zeros_like(r)
    running_add = 0

    for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # if the game ended (in Pong), reset 
    running_add = running_add * gamma + r[t] 
    discounted_r[t] = running_add
    discounted_r -= np.mean(discounted_r) #normalizing the result
    discounted_r /= np.std(discounted_r) #idem using standar deviation
    return discounted_r

In [ ]:
model = Sequential()# hidden layer takes a pre-processed frame as input, and has 200 units. Simple layer architectur of 200 x1, 1x1
model.add(Dense(units=200,input_dim=80*80, activation='relu', kernel_initializer='glorot_uniform'))# output layer — we use a Sigmoid here, in order to get a 0, or 1 value to represent ACTION UP
model.add(Dense(units=1, activation='sigmoid', kernel_initializer='RandomNormal'))# compile the model using traditional Machine Learning losses and optimizers
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
history = []
observation = env.reset()
prev_input = None
UP_ACTION = 2
DOWN_ACTION = 3

gamma = 0.99# initialization of variables used in the main loop
x_train, y_train, rewards = [],[],[]
reward_sum = 0
episode_nb = 0

it=0

# main training loop
while (True):
    cur_input = prepro(observation)
    #print(len(cur_input)) — Sanity Check reasons only

    x = cur_input - prev_input if prev_input is not None else np.zeros(80 * 80)
    prev_input = cur_input

    # forward the policy network and sample action according to the probability distribution

    proba = model.predict(np.expand_dims(x, axis=1).T)


    action = UP_ACTION if np.random.uniform() < proba else DOWN_ACTION
    y = 1 if action == 2 else 0 # 0 and 1 are our labels# log the input and label to train later
    x_train.append(x)
    y_train.append(y)# do one step in our environment — This is returned by our environment in OpenAI gym. 
    observation, reward, done, info = env.step(action)
    env.render()

    rewards.append(reward)
    reward_sum += reward
    
    if done:
        it+=1
        print("done {}-times".format(it))
        history.append(reward_sum)
        print(‘At the end of episode’, episode_nb, ‘the total reward was :’, reward_sum)
        if episode_nb>=3000 and reward_sum >=-12:
        break
        else:

        # increment episode number
        episode_nb += 1

        # training
        model.fit(x=np.vstack(x_train), y=np.vstack(y_train), verbose=1, sample_weight=discount_rewards(rewards, gamma))


        # Reinitialization
        x_train, y_train, rewards = [],[],[]
        observation = env.reset()
        reward_sum = 0
        prev_input = None

In [61]:
#buildt model: input layer -> 2x fully connected -> output layer
#The input to the neural
#network consists is an 84 ×84 ×4 image produced by φ. The first hidden layer convolves 16 8 ×8
#filters with stride 4 with the input image and applies a rectifier nonlinearity [10, 18]. The second
#hidden layer convolves 32 4 ×4 filters with stride 2, again followed by a rectifier nonlinearity. The
#final hidden layer is fully-connected and consists of 256 rectifier units. The output layer is a fully-
#connected linear layer with a single output for each valid action. The number of valid actions varied
#between 4 and 18 on the games we considered. We refer to convolutional networks trained with our
#approach as Deep Q-Networks (DQN). [Playing Atari with Deep Reinforcement Learning] 
#I will do it with one image bc its quite late and I want to leave something to do for another day:)
def build_model(states, actions):
    model = Sequential()
    model.add(Conv2D(32, (8, 8),input_shape=(84,84,3), strides = 4, activation = 'relu'))
    model.add(Conv2D(64, (4, 4), strides = 2, activation = 'relu'))
    model.add(Conv2D(64, (3, 3), strides = 1, activation = 'relu'))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    #model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    model.summary()
    return model

In [62]:
model = build_model(states, actions)

Exception ignored in: <function SimpleImageViewer.__del__ at 0x0000018F60896AF0>
Traceback (most recent call last):
  File "C:\Users\erics\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\classic_control\rendering.py", line 458, in __del__
    self.close()
  File "C:\Users\erics\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\classic_control\rendering.py", line 454, in close
    self.window.close()
  File "C:\Users\erics\AppData\Local\Programs\Python\Python39\lib\site-packages\pyglet\window\win32\__init__.py", line 328, in close
    super(Win32Window, self).close()
  File "C:\Users\erics\AppData\Local\Programs\Python\Python39\lib\site-packages\pyglet\window\__init__.py", line 857, in close
    app.windows.remove(self)
  File "C:\Users\erics\AppData\Local\Programs\Python\Python39\lib\_weakrefset.py", line 110, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x0000018F681B5E50; to 'Win32Window' at 0x0000018F5DDEA6A0>


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 20, 20, 32)        6176      
                                                                 
 conv2d_12 (Conv2D)          (None, 9, 9, 64)          32832     
                                                                 
 conv2d_13 (Conv2D)          (None, 7, 7, 64)          36928     
                                                                 
 flatten_6 (Flatten)         (None, 3136)              0         
                                                                 
 dense_10 (Dense)            (None, 512)               1606144   
                                                                 
 dense_11 (Dense)            (None, 6)                 3078      
                                                                 
Total params: 1,685,158
Trainable params: 1,685,158
No

In [63]:
#gym wrapper
class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        
def observation(self, obs):
    return ProcessFrame84.process(obs)

@staticmethod
def process(frame):
    if frame.size == 210 * 160 * 3:
        img = np.reshape(frame, [210, 160,  3]).astype(np.float32)
    elif frame.size == 250 * 160 * 3:
        img = np.reshape(frame, [250, 160, 3]).astype(np.float32)
    else:
        assert False, "Unknown resolution."      
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 +  img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)

In [64]:
class BufferWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_steps, dtype=np.float32):
        super(BufferWrapper, self).__init__(env)
        self.dtype = dtype
        old_space = env.observation_space
        self.observation_space = gym.spaces.Box(old_space.low.repeat(n_steps, axis=0),old_space.high.repeat(n_steps, axis=0), dtype=dtype)
        
    def reset(self):
        self.buffer = np.zeros_like(self.observation_space.low,
        dtype=self.dtype)
        return self.observation(self.env.reset())
    
    def observation(self, observation):
        #self.buffer[:-1] = self.buffer[1:]
        #self.buffer[-1] = observation
        #return self.buffer
        return prepro(observation)

In [65]:
class ScaledFloatFrame(gym.ObservationWrapper):
     def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0

In [66]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=5000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [67]:
env = BufferWrapper(gym.make('Pong-v0'), 1)

In [68]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
#with tf.device('/gpu:0'): #please use my GPU if it would recognise it :(
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected conv2d_11_input to have 4 dimensions, but got array with shape (1, 1, 6400)